In [1]:
%load_ext autoreload
%autoreload 2
%run ~/FKMC/notebooks/notebook_preamble.py
%matplotlib inline
np.seterr(under = 'ignore')
from time import time
from munch import munchify

Imported pathlib::Path, os, munch::Munch, pickle, logging, itertools::count, matplotlib::colors
Names are pyplot = plt, matplotlib = mpl, numpy = np


In [12]:
from FKMC.import_funcs import get_data_funcmap_chain_ext, mean_over_MCMC, IPRandDOS, extract

get_data_funcmap_chain_ext(Path('~/HPC_data/Tsweep_U5_J5'),
            functions = [IPRandDOS(E_bins = np.linspace(-20, 20, 10000 + 1)), 
                             mean_over_MCMC('Mf_moments'),
                            ],
            structure_names = ('Ts',),
            structure_dims = (),
            )

[PosixPath('/Users/tom/HPC_data/Tsweep_U5_J5/code/varyingT.py')]
Ham_params:  t=1, alpha=1.25, mu=0, beta=varying, J=5, U=5, normalise=True,


Tasks per chain: 10,

Each doing 1000 steps,

10000 total chain length,

1000 samples,


MCMC_params:  N_steps=1000, N_burn_in=1000, thin=10, logger=<FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x7f91dc35e8d0>, proposal=<function p_multi_site_uniform_reflect at 0x7f91dc223950>, accept_function=<function perturbation_accept at 0x7f91dc223cb0>, warnings=False,


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()